In [2]:
import os
import numpy as np
import pandas as pd  # For saving results as a .csv file
import sep
import h5py
import scipy.ndimage as ndimage
from photutils.isophote import Ellipse, EllipseGeometry
from tqdm import tqdm

# Function to analyze a single image with SEP and return the metrics
def analyze_image_with_sep(image, apply_smoothing=True, sigma=1.5, noise_threshold=0.50):
    result = {}
    
     if image.ndim == 3:
        image = image[0]  # Use the first channel

    if apply_smoothing:
        image = ndimage.gaussian_filter(image, sigma=sigma)

    bkg = sep.Background(image)
    image_sub = image - bkg

    if np.isnan(image_sub).all() or image_sub.size == 0:
        return None

    mean_snr = np.mean(image_sub / bkg.globalrms)
    if np.isnan(mean_snr) or mean_snr < noise_threshold:
        return None

    objects = sep.extract(image_sub, 1.5, err=bkg.globalrms)
    if len(objects) == 0:
        return None

     
    obj = objects[0]
    x = obj['x']
    y = obj['y']
    semi_major_axis = obj['a']
    semi_minor_axis = obj['b']
    orientation_angle = obj['theta']
    isophotal_area = obj['npix']
    ellipticity = 1 - (semi_minor_axis / semi_major_axis)

    result['Semi-major Axis'] = semi_major_axis
    result['Semi-minor Axis'] = semi_minor_axis
    result['Ellipticity'] = ellipticity
    result['Orientation Angle'] = orientation_angle
    result['Isophotal Area'] = isophotal_area

    if semi_major_axis > 0 and 0 <= ellipticity <= 1:
        try:
            geometry = EllipseGeometry(x0=x, y0=y, sma=semi_major_axis, eps=ellipticity, pa=orientation_angle)
            ellipse = Ellipse(image_sub, geometry)
            isophote_list = ellipse.fit_image()
            
            # Estimate Sersic index using the isophote data
            sersic_n = np.log(2) / np.log((1 + ellipticity) / (1 - ellipticity))
            result['Sersic Index'] = sersic_n
        except Exception as e:
            print(f"  Error fitting Sersic profile: {e}")
            result['Sersic Index'] = np.nan
    
    return result

# Path to the HDF5 file
hdf5_file_path = './ddpm_5x64x64_testing.hdf5'

# Open the HDF5 file
with h5py.File(hdf5_file_path, 'r') as f:
    # key in HDF5 where images are stored
    images_dataset = f['image']
    
    # List to store results
    results = []

    # Process and analyze each image
    with tqdm(total=images_dataset.shape[0], desc="Processing images from HDF5") as pbar:
        for i in range(images_dataset.shape[0]):
            image = images_dataset[i]
            metrics = analyze_image_with_sep(image)
            if metrics:
                # Add a reference to the image 
                metrics['Image Index'] = i
                results.append(metrics)

            pbar.update(1)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Path to save the .csv file
csv_path = './AndrewMetrics.csv'

# Ensure the directory exists; if not, create it
os.makedirs(os.path.dirname(csv_path), exist_ok=True)

# Save the DataFrame to a .csv file
results_df.to_csv(csv_path, index=False)

print(f"Results saved to {csv_path}")


Processing images from HDF5:   0%|          | 4/40914 [00:01<3:24:33,  3.33it/s]/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


  Error fitting Sersic profile: list index out of range


Processing images from HDF5:   0%|          | 63/40914 [00:17<3:40:03,  3.09it/s]/home/jupyter-jacob/.local/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Processing images from HDF5:   3%|▎         | 1392/40914 [07:28<3:54:16,  2.81it/s]

  Error fitting Sersic profile: index 0 is out of bounds for axis 0 with size 0


Processing images from HDF5:   4%|▍         | 1552/40914 [08:26<3:14:27,  3.37it/s]WARNING: Improper input: func input vector length N=5 must not exceed func output vector length M=4 [photutils.isophote.fitter]


...

  Error fitting Sersic profile: list index out of range


Processing images from HDF5:  98%|█████████▊| 40003/40914 [2:57:33<07:22,  2.06it/s]WARNING: Improper input: func input vector length N=5 must not exceed func output vector length M=3 [photutils.isophote.fitter]


  Error fitting Sersic profile: list index out of range


Processing images from HDF5: 100%|██████████| 40914/40914 [3:02:00<00:00,  3.75it/s]


Results saved to ./AndrewMetrics.csv
